In [2]:
from __future__ import print_function
import sys
import json
import os
import subprocess
import re
import string
from bs4 import BeautifulSoup
import urllib2
import csv

In [3]:
collection = "gov2"

projectDir = "/home/fj9124/projects/ir/seq_kb_ir/"
optParamsDir = os.path.join(projectDir, "optParams")
annotateProg = os.path.join(optParamsDir, "annotate.py")
print("annotateProg =", annotateProg)

configsDir = os.path.join(projectDir, "configs")
colConfigsDir = os.path.join(configsDir, collection)
if collection == "gov2":
    colInConfigsFile = os.path.join(colConfigsDir, "06.topics.701-850.txt")
print ("colInConfigsFile =", colInConfigsFile)
colOutConfigsFile = os.path.join(colConfigsDir, "indriRunQuery.cfg")
print ("colOutConfigsFile =", colOutConfigsFile)

indexDir = "/scratch/index/indri_5_7/"
colIndexDir = os.path.join(indexDir, collection) 
print("collection index Dir =", colIndexDir)

colRunsFile = os.path.join(projectDir, "runs", collection, "indriRunQuery.run")
print("colRunsFile =", colRunsFile)

if collection == "gov2":
    colQrelsFile = os.path.join(projectDir, "qrels", collection, "qrels.tb06.top50")
print("colQrelsFile =", colQrelsFile)

colEvalsFile = os.path.join(projectDir, "evals", collection, "indriRunQuery.evals")
print("colEvalsFile =", colEvalsFile)

annotateProg = /home/fj9124/projects/ir/seq_kb_ir/optParams/annotate.py
colInConfigsFile = /home/fj9124/projects/ir/seq_kb_ir/configs/gov2/06.topics.701-850.txt
colOutConfigsFile = /home/fj9124/projects/ir/seq_kb_ir/configs/gov2/indriRunQuery.cfg
collection index Dir = /scratch/index/indri_5_7/gov2
colRunsFile = /home/fj9124/projects/ir/seq_kb_ir/runs/gov2/indriRunQuery.run
colQrelsFile = /home/fj9124/projects/ir/seq_kb_ir/qrels/gov2/qrels.tb06.top50
colEvalsFile = /home/fj9124/projects/ir/seq_kb_ir/evals/gov2/indriRunQuery.evals


In [44]:
#example: origText = "Relevant documents will include those on historical exploration and drilling as well as history of regulatory bodies. Relevant are history of the oil industry in various states, even if drilling began in 1950 or later"
def annotateText(origText):
    #print("origText =", origText)
    cmd = ' '.join([annotateProg, "\"" + origText + "\""])
    #print("cmd =", cmd)
    annotateResults = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE).stdout.read()
    #print("annotateResults =", annotateResults)
    #annotateResults = subprocess.check_output([annotateProg, "\"" + origText + "\""])
    print("annotateResults =", annotateResults[0:500], "...")
    annotateResultsJson = re.split('## Response Object ##|## Keywords ##',annotateResults)[1].strip()
    annotateResultsJsonParsed = json.loads(annotateResultsJson)
    freebaseEntities = [ j[u'freebase'] for j in annotateResultsJsonParsed["concepts"] if u'freebase' in j]
    #print (freebaseEntities)
    entityTexts = [ str(j['text'].encode('utf8')) for j in annotateResultsJsonParsed["concepts"]]
    #print (entityTexts)
    return(entityTexts)

In [5]:
# testing annotation:
origText = "Relevant documents will include those on historical exploration and drilling as well as history of regulatory bodies. Relevant are history of the oil industry in various states, even if drilling began in 1950 or later"
annotateText(origText)

origText = Relevant documents will include those on historical exploration and drilling as well as history of regulatory bodies. Relevant are history of the oil industry in various states, even if drilling began in 1950 or later


['Petroleum', 'Renaissance', 'History', 'Petroleum industry']

In [ ]:
#print("annotateResults =", annotateResults[0:500], "...")

In [ ]:
#annotateResultsJson = re.split('## Response Object ##|## Keywords ##',annotateResults)[1].strip()

In [ ]:
#annotateResultsJsonParsed = json.loads(annotateResultsJson)
#freebaseEntities = [ j[u'freebase'] for j in annotateResultsJsonParsed["concepts"]]
#print (freebaseEntities)
#entityTexts = [ str(j['text']) for j in annotateResultsJsonParsed["concepts"]]
#print (entityTexts)

In [6]:
print("extracting keys from /wikipedia/en namespace of freespace")
print ("freebase url =", freebaseEntities[0])
html_doc = urllib2.urlopen(freebaseEntities[0])
soup = BeautifulSoup(html_doc, "lxml")

freebaseContent = soup.text.split('\n')

#  "the key [that] can be used to formulate a URL to the corresponding English article on Wikipedia"
keyWikipediaEn = "key:wikipedia.en "

wikipediaEn = [re.sub(keyWikipediaEn, '', str(i)).strip() for i in freebaseContent if keyWikipediaEn in i]
wikipediaEn = [re.sub('[^a-zA-Z]+', ' ', i) for i in wikipediaEn]
wikipediaEn = [re.sub( '\s+', ' ', i ).strip() for i in wikipediaEn]
print("wikipediaEn =", wikipediaEn)


#keyWikipediaObjectKeyEn = "ns:type.object.key    ns:wikipedia.en."

#wikipediaObjectKeyEn = [re.sub(keyWikipediaObjectKeyEn, '', str(i)).strip() for i in freebaseContent if keyWikipediaObjectKeyEn in i]
#wikipediaObjectKeyEn = [re.sub('[^a-zA-Z]+', ' ', i) for i in wikipediaObjectKeyEn]
#wikipediaObjectKeyEn = [re.sub( '\s+', ' ', i ).strip() for i in wikipediaObjectKeyEn]
#print("wikipediaObjectKeyEn =", wikipediaObjectKeyEn)

#print(set(wikipediaObjectKeyEn) - set(wikipediaEn))



extracting keys from /wikipedia/en namespace of freespace


NameError: name 'freebaseEntities' is not defined

In [18]:
print("parse queries file")
regex = re.compile('[%s]' % re.escape(string.punctuation))
queries = dict()
with open(colInConfigsFile, 'r') as infile:
    xmlDoc = infile.read()
    soup = BeautifulSoup(xmlDoc, "lxml")
    topTags = soup.findAll('top')
    #print(soup.prettify())
    for topTag in topTags:
        numTag = topTag.findAll('num')[0]
        qNo = int(str(numTag.contents[0]).replace("Number: ", ""))
        if qNo < 801:
            continue
        #narrTag = numTag.findAll('title')[0].findAll('desc')[0].findAll('narr')[0]
        #qText = str(numTag.findAll('title')[0].contents[0])
        qText = str(numTag.findAll('title')[0].findAll('desc')[0].contents[0])
        #print(qTextTag)
        qText = qText.replace("Description:","").replace("\n", " ").strip()
        #narr = narrTag.string.replace("Narrative:","").replace("\n", " ").strip()
        qText = regex.sub(' ', qText)
        qText = re.sub( '\s+', ' ', qText)
        queries[qNo] = qText
print ("queries =", queries)

parse queries file
queries = {801: 'Describe the origin nature extent of spread and means of controlling kudzu ', 802: 'What is the impact of volcano eruptions on global temperature ', 803: 'Give the history of this holiday and the various ways of celebrating May Day ', 804: 'Describe resolutions proposed and legislation passed to ban the cloning of humans and the rationale for the bans ', 805: 'Describe the Identify Theft Passport issued to identity theft victims to show to creditors and law enforcement officers questioning their credit worthiness or innocence ', 806: 'What is Doctors Without Borders Medecins Sans Frontieres and what do they do ', 807: 'Describe the nature and history of sugar tariff rate quotas in the United States ', 808: 'What information is available on the involvement of the North Korean Government in counterfeiting of US currency ', 809: 'Identify wastewater treatment projects that involve constructed or natural wetlands ', 810: 'Provide information regarding ti

In [41]:
def mrfQueryGen(conceptText, odW, uwW):
    mrfText = "#weight( "
    mrfText += str(1 - odW - uwW) + " #combine( " + conceptText + " ) "
    conceptText_ = conceptText.split(" ")
    #if len(conceptText_) > 2:
    odText = ""
    uwText = ""
    for i in range(1, len(conceptText_)):
        odText += " #od4( " + ' '.join(conceptText_[i-1:i+1]) + " ) "
        uwText += " #uw17( " + ' '.join(conceptText_[i-1:i+1]) + " ) "
    if len(odText) > 10: # if it is not just tags
        mrfText += str(odW) + " #combine( " + odText + " ) "
        mrfText += str(uwW) + " #combine( " + uwText + " ) "
    mrfText += " ) "
    return(mrfText)

In [39]:
def generateQuery(dirCoeff, intCoeff, odW, uwW):
    soupNew = BeautifulSoup("", 'lxml')
    parameters_tag = soupNew.new_tag("parameters")
    index_tag = soupNew.new_tag("index")
    index_tag.string = colIndexDir
    parameters_tag.append(index_tag)
    for qNo, qText in queries.iteritems():
        doc_tag = soupNew.new_tag("query")

        docno_tag = soupNew.new_tag("number")
        docno_tag.string = str(qNo)
        doc_tag.append(docno_tag)
        
        text_tag = soupNew.new_tag("text")
        
        text_tag.string = "#weight( "
        text_tag.string += str(1- intCoeff) + " #combine( " + qText + ")"
        
        concepts = annotateText(qText)
        
        text_tag.string += str(intCoeff) + " #combine("
        for concept in concepts:
            print("concept =", concept.decode('utf-8'))
            text_tag.string += mrfQueryGen(concept.decode('utf-8'), odW, uwW)
        text_tag.string += " )"

        text_tag.string += " )"
        
        doc_tag.append(text_tag)

        parameters_tag.append(doc_tag)
        #print(doc_tag)

    rule_tag = soupNew.new_tag("rule")
    rule_tag.string = "method:dir,mu:" + str(dirCoeff)
    #rule_tag.string = "method:two"
    parameters_tag.append(rule_tag)

    #intCoeff_tag = soupNew.new_tag("intCoeff")
    #intCoeff_tag.string = "0.8"
    #parameters_tag.append(intCoeff_tag)

    threads_tag = soupNew.new_tag("threads")
    threads_tag.string = "32"
    parameters_tag.append(threads_tag)

    count_tag = soupNew.new_tag("count")
    count_tag.string = "1000"
    parameters_tag.append(count_tag)

    trecFormat_tag = soupNew.new_tag("trecFormat")
    trecFormat_tag.string = "true"
    parameters_tag.append(trecFormat_tag)

    soupNew.append(parameters_tag)
    print(soupNew.prettify())
    #print("outFileName =", outFileName)

    with open(colOutConfigsFile, 'w') as outfile:
        outfile.write(str(soupNew))

In [45]:
dirrCoeff= 2600
intCoeff = 0.8
odW = 0.15
uwW = 0.15
generateQuery(dirrCoeff, intCoeff, odW, uwW)

annotateResults = text = Describe the origin nature extent of spread and means of controlling kudzu 
Error in combined call:  daily-transaction-limit-exceeded
 ...


IndexError: list index out of range

In [ ]:
cmd = ' '.join(["IndriRunQuery", colOutConfigsFile, ">", colRunsFile])
print("cmd =", cmd)
subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE).stdout.read()

In [ ]:
cmd = ' '.join(["trec_eval", "-q", colQrelsFile, colRunsFile, ">", colEvalsFile])
print("cmd =", cmd)
subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE).stdout.read()

In [ ]:
with open(colEvalsFile, "r") as infile:
    reader = csv.reader(infile, delimiter='\t')
    precTable = list(reader)
    #print(precTable)
    precTable = {i[0].strip(): float(i[2].strip()) for i in precTable if i[1].strip()=="all" and i[0].strip() != "runid"}
    print(precTable['map'])